In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
import seaborn as sns
from datetime import datetime
from utils.neural_network import train_nn, get_device, batch_data_by_country

from utils.load_data import load_data, load_gt_data
from utils.preprocessing_v2 import Preprocessing, get_gt_diff_logs
import utils.results as results
import torch
import torch.nn as nn

TRENDS_FOLDER = 'data/google_trends/'
GDP_FOLDER = 'data/gdp/'
DATA_PREFIX = 'trends_data_by_topic_'

EPS = 1e-15
SEED = 42

## Loading and Preprocessing Data

### Loading Google Trends

In [ ]:
TRAIN_PROPORTION = 0.9
MODE = "pct" # None | "pct" | "diff"
PERIOD = 4  # Year to year prediction
device = get_device()

In [ ]:
data, all_gdps, all_gts = load_data()
data['country'].unique()

In [ ]:
preprocessor = Preprocessing(epsilon=EPS, gdp_diff_period=PERIOD, all_GDPs=all_gdps, all_GTs=all_gts)

X_train, y_train, X_valid, y_valid, x_hf = preprocessor.preprocess_data(train_pct=TRAIN_PROPORTION, 
                                                                  mode="diff", 
                                                                  take_log_diff_gdp=True,
                                                                  gt_trend_removal=False, 
                                                                  keep_pca_components=180, 
                                                                  noisy_data_stds=[0.005, 0.05, 0.1], 
                                                                  add_encoded_month=False, 
                                                                  gt_data_transformations=[get_gt_diff_logs], other_params={'plot_pca': False})

x_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
x_valid_t = torch.tensor(X_valid, dtype=torch.float32).to(device)
y_valid_t = torch.tensor(y_valid, dtype=torch.float32).to(device)
x_hf_t = torch.tensor(x_hf, dtype=torch.float32).to(device)

print(all_gts['country'].unique())

## Smoothing Model

In [ ]:
preprocessor.dates_high_freq
preprocessor.country_high_freq

df_hf = pd.DataFrame({
    'date': preprocessor.dates_high_freq,
    'country': preprocessor.country_high_freq,
    'data': [x_hf[i] for i in range(len(x_hf))]
})

df_x_train = pd.DataFrame({
    'date': preprocessor.dates_train.copy(),
    'country': preprocessor.country_train.copy(),
    'data': [X_train[i].copy() for i in range(len(X_train))],
    'y_true': [y_train[i].copy() for i in range(len(y_train))]
})

df_hf['date'] = pd.to_datetime(df_hf['date'])
df_x_train['date'] = pd.to_datetime(df_x_train['date'])

In [ ]:
from torch.nn import MSELoss

def smoothness_loss(y_pred, smoothness_weight_ord1=0.33, smoothness_weight_ord2=0.33):
    if y_pred.shape[1] > 1:
        # Compute the loss related to smoothness
        smoothness_loss_ord1 = 0
        for i in range(1, 4):
            # Divide by 3 because dt is 1/3 month, and by 4 because we have 4 time points and we average them
            smoothness_loss_ord1 += torch.linalg.norm((y_pred[:, i, :] - y_pred[:, i - 1, :])) ** 2 / (3 * 4 * y_pred.shape[0])

        # Constraint on the double derivative
        smoothness_loss_ord2 = 0
        for i in range(2, 4):
            smoothness_loss_ord2 += torch.linalg.norm(y_pred[:, i, :] - 2 * y_pred[:, i - 1, :] + y_pred[:, i - 2, :]) ** 2 / (3 * 4 * y_pred.shape[0])

        return smoothness_loss_ord1 * smoothness_weight_ord1 + smoothness_loss_ord2 * smoothness_weight_ord2

    else:
        raise ValueError("Input shape not supported!")

def custom_loss(y_pred, y_true, smoothness_weight_ord1=0.33, smoothness_weight_ord2=0.33):
    # Compute usual MSE on actual GDP values
    mse = MSELoss()

    if y_pred.shape[1] > 1:
        prediction_loss = mse(y_pred[:, 4, :], y_true)  # First column is the actual GDP

        smooth_loss = smoothness_loss(y_pred, smoothness_weight_ord1, smoothness_weight_ord2)

        return prediction_loss + smooth_loss
    elif y_pred.shape[1] == 1:
        return mse(y_pred, y_true)
    else:
        raise ValueError("Input shape not supported!")

In [ ]:
augmented_data, filtered_countries, filtered_dates, y_augmented_train = batch_data_by_country(df_x_train, 
                                                                                              df_hf, 
                                                                                              4, 
                                                                                              0, 
                                                                                              ['y_true'], 
                                                                                              False)

y_augmented_train = np.array(y_augmented_train).squeeze().astype(np.float32)

In [ ]:
from tqdm.notebook import tqdm

training_losses = []
validation_losses = []
validation_r_squareds = []
ord1_smoothness_weights = np.logspace(-1, 2, 10)
ord2_smoothness_weights = np.logspace(2, 3.2, 10)
smoothness_losses = []
global_mse_losses = []
parameters = []

for ord1_weight in tqdm(ord1_smoothness_weights):
    for ord2_weight in tqdm(ord2_smoothness_weights, leave=False):
        model, training_loss, validation_loss, validation_r_squared, mse_loss = train_nn(augmented_data, 
                                                                            y_augmented_train, 
                                                                            X_valid, 
                                                                            y_valid, 
                                                                            num_epochs=100, 
                                                                            learning_rate=5e-4, 
                                                                            weight_decay=5e-2, 
                                                                            verbose=False,
                                                                            custom_loss=lambda x, y: custom_loss(x, y, ord1_weight, smoothness_weight_ord2=ord2_weight),
                                                                            current_gdp_idx=4,
                                                                            seed=SEED)
        parameters.append((ord1_weight, ord2_weight))
        training_losses.append(training_loss)
        validation_losses.append(validation_loss)
        validation_r_squareds.append(validation_r_squared)
        global_mse_losses.append(mse_loss)

        y_pred_hf = model(torch.tensor(x_hf, dtype=torch.float32).to(device)).clone().detach().cpu().numpy().squeeze()

        df_preds = pd.DataFrame({
            'date': preprocessor.dates_high_freq,
            'country': preprocessor.country_high_freq,
            'data': [y_pred_hf[i] for i in range(len(x_hf))]
        })

        smoothness_values = []
        for country in df_preds['country'].unique():
            country_data = df_preds[df_preds['country'] == country]
            
            smoothness_values_per_func = []
            for smoothness_func in results.all_smoothness_metrics:
                smoothness_values_per_func.append(smoothness_func(country_data['data'].values))

            smoothness_values.append(smoothness_values_per_func)

        smoothness_values = np.array(smoothness_values)

        # Store different means of the smoothness values
        smoothness_losses.append([np.mean(smoothness_values), np.expm1(np.mean(np.log1p(smoothness_values))), len(smoothness_values) / np.sum(1 / smoothness_values)])

In [ ]:
def plot_heatmap_top_filtered_percent(data, x_vals, y_vals, title, xlabel, ylabel, cmap='viridis', threshold=5):
    # Compute the 90th percentile threshold
    valid_data = data[np.isfinite(data)]
    if len(valid_data) == 0:
        print("No valid data to plot.")
        return
    #threshold = np.percentile(valid_data, 90)
    
    # Mask values below the threshold
    data_filtered = np.where((data >= -threshold) & (data <= threshold), data, np.nan)
    
    im = plt.imshow(data_filtered, origin='lower', 
                    extent=[min(x_vals), max(x_vals), min(y_vals), max(y_vals)], 
                    aspect='auto', cmap=cmap) #, norm=norm)
    cbar = plt.colorbar(im)
    cbar.set_label('Metric Value')
    plt.title(title + f' (Between -{threshold} and {threshold})')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xscale('log')
    plt.yscale('log')


In [ ]:
N1 = len(ord1_smoothness_weights)
N2 = len(ord2_smoothness_weights)

# Create mapping from (ord1_weight, ord2_weight) to grid indices
ord1_to_idx = {w: i for i, w in enumerate(ord1_smoothness_weights)}
ord2_to_idx = {w: i for i, w in enumerate(ord2_smoothness_weights)}

# Initialize empty arrays for final metrics
final_train_loss = np.zeros((N1, N2))
final_val_loss = np.zeros((N1, N2))
final_val_r2 = np.zeros((N1, N2))
final_smoothness_mean = np.zeros((N1, N2))
final_validation_mse_loss = np.zeros((N1, N2))  # example if needed
final_training_mse_loss = np.zeros((N1, N2))  # example if needed

for (ord1, ord2), tr_loss, val_loss, val_r2, mse_loss, sm_loss in zip(parameters, 
                                                                    training_losses, 
                                                                    validation_losses, 
                                                                    validation_r_squareds, 
                                                                    global_mse_losses, 
                                                                    smoothness_losses):
    i = ord1_to_idx[ord1]
    j = ord2_to_idx[ord2]

    final_train_loss[i, j] = tr_loss[-1] if len(tr_loss) > 0 else np.nan
    final_val_loss[i, j] = val_loss[-1] if len(val_loss) > 0 else np.nan
    final_val_r2[i, j] = val_r2[-1] if len(val_r2) > 0 else np.nan
    
    final_smoothness_mean[i, j] = sm_loss[1] if len(sm_loss) > 0 else np.nan
    final_validation_mse_loss[i, j] = mse_loss[-1][1] if len(mse_loss) > 0 else np.nan
    final_training_mse_loss[i, j] = mse_loss[-1][0] if len(mse_loss) > 0 else np.nan

plt.figure(figsize=(10, 8))

# Plot final training loss
plt.subplot(2, 2, 1)
plot_heatmap_top_filtered_percent(-final_train_loss, ord1_smoothness_weights, ord2_smoothness_weights, 
             title='Final Training Loss', 
             xlabel='ord1_weight', 
             ylabel='ord2_weight')

plt.subplot(2, 2, 2)
# Example usage with final_validation_loss (or any other metric you computed):
plot_heatmap_top_filtered_percent(-final_val_loss, ord1_smoothness_weights, ord2_smoothness_weights, 
                            title='Final Validation Loss', 
                            xlabel='ord1_weight', 
                            ylabel='ord2_weight')

plt.subplot(2, 2, 3)
plot_heatmap_top_filtered_percent(final_val_r2, ord1_smoothness_weights, ord2_smoothness_weights, 
             title='Final Validation - R^2', 
             xlabel='ord1_weight', 
             ylabel='ord2_weight')

# Plot final smoothness mean metric
plt.subplot(2, 2, 4)
plot_heatmap_top_filtered_percent(-final_smoothness_mean, ord1_smoothness_weights, ord2_smoothness_weights, 
             title='Final Smoothness (Mean)', 
             xlabel='ord1_weight', 
             ylabel='ord2_weight')

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plot_heatmap_top_filtered_percent(-final_training_mse_loss, ord1_smoothness_weights, ord2_smoothness_weights, 
             title='Final Training MSE Loss', 
             xlabel='ord1_weight', 
             ylabel='ord2_weight')

plt.subplot(1, 2, 2)
plot_heatmap_top_filtered_percent(-final_validation_mse_loss, ord1_smoothness_weights, ord2_smoothness_weights, 
             title='Final Validation MSE Loss', 
             xlabel='ord1_weight', 
             ylabel='ord2_weight')
plt.tight_layout()
plt.show()


In [ ]:
smoothness_weight_ord1=40
smoothness_weight_ord2=40
model, training_loss, validation_loss, validation_r_squared, global_mse_losses = train_nn(augmented_data, 
                                                                        y_augmented_train, 
                                                                        X_valid, 
                                                                        y_valid, 
                                                                        num_epochs=100, 
                                                                        learning_rate=5e-4, 
                                                                        weight_decay=5e-2, 
                                                                        verbose=True,
                                                                        current_gdp_idx=4,
                                                                        custom_loss=lambda x, y: custom_loss(x, y, smoothness_weight_ord1, smoothness_weight_ord2),
                                                                        seed=SEED)

In [ ]:
device = get_device(False)

# Get the predictions
y_pred = model(x_valid_t).clone().detach().cpu().numpy().squeeze()
y_pred_train = model(x_train_t).clone().detach().cpu().numpy().squeeze()
y_pred_hf = model(x_hf_t).clone().detach().cpu().numpy().squeeze()

In [ ]:

# Associate the result by country and add 'Set' column
y_pred_country = pd.DataFrame({
    'date': preprocessor.dates_valid,
    'country': preprocessor.country_valid,
    'y_pred': y_pred,
    'y_true': y_valid,
    'Set': 'Validation'
})

y_pred_train_country = pd.DataFrame({
    'date': preprocessor.dates_train,
    'country': preprocessor.country_train,
    'y_pred': y_pred_train,
    'y_true': y_train,
    'Set': 'Training'
})

# Put together the train and the validation set
predictions = pd.concat([y_pred_train_country, y_pred_country])
predictions_hf = pd.DataFrame({
    'date': preprocessor.dates_high_freq,
    'country': preprocessor.country_high_freq,
    'y_pred': y_pred_hf,
    'Set': 'High Frequency'
})


# Melting the dataframe for better plotting
predictions_melted = predictions.melt(
    id_vars=["date", "country", "Set"],
    value_vars=["y_pred", "y_true"], 
    var_name="Type", value_name="Value"
)

predictions_hf_melted = predictions_hf.melt(
    id_vars=["date", "country", "Set"],
    value_vars=["y_pred"], 
    var_name="Type", value_name="Value"
)

# Function to plot data for the selected country
def plot_by_country(selected_country):
    filtered_data = predictions_melted[predictions_melted["country"] == selected_country]
    filtered_data_hf = predictions_hf_melted[predictions_hf_melted["country"] == selected_country]

    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=filtered_data_hf,
        x="date", y="Value", hue="Type", errorbar = None, style="Set", markers=True, palette="Set3"
    )
    sns.lineplot(
        data=filtered_data,
        x="date", y="Value", hue="Type", errorbar = None, style="Set", markers=True
    )
    
    plt.title(f"Prediction vs True Values for {selected_country}")
    plt.xlabel("Date")
    plt.ylabel("Values")
    plt.legend(title="Legend", loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(True)
    plt.tight_layout()
    plt.show()


# Create a dropdown widget for selecting the country
countries = predictions["country"].unique()
dropdown = widgets.Dropdown(
    options=countries,
    value=countries[0],
    description='Country:'
)

# Use the interact function to link the dropdown with the plot function
interact(plot_by_country, selected_country=dropdown)

plt.show()

In [ ]:
y_pred_hf

In [ ]:
# Compute per country
smoothness_values = []
for country in df_preds['country'].unique():
    country_data = df_preds[df_preds['country'] == country]
    
    smoothness_values_per_func = []
    for smoothness_func in results.all_smoothness_metrics:
        smoothness_values_per_func.append(smoothness_func(country_data['data'].values))

    smoothness_values.append(smoothness_values_per_func)

smoothness_values = np.array(smoothness_values)

plt.barh(range(smoothness_values.shape[1]), np.log(smoothness_values.mean(axis=0)))
plt.show()

In [ ]:
np.mean(smoothness_values), np.exp(np.mean(np.log(smoothness_values))), len(smoothness_values) / np.sum(1 / smoothness_values)

### Case Study: Fourrier Transform Smoothing

In [ ]:
def fft_analysis(preds, cutoff=0.05): 
    fft = np.fft.fft(preds)
    freqs = np.fft.fftfreq(len(preds))

    filtered = np.copy(fft)
    filtered[np.abs(freqs) > cutoff] = 0

    y_filtered = np.fft.ifft(filtered)
    plt.plot(preds, label='Original')
    plt.plot(y_filtered, label='Filtered')
    plt.legend()
    plt.show()

# Per country
for country in df_preds['country'].unique():
    country_data = df_preds[df_preds['country'] == country]
    fft_analysis(country_data['data'].values)